In [36]:
import pandas as pd
import numpy as np


csv_file = '../Data_Cleanup/outCSV/Station_ID_with_AvgIdleDaytimesDays_and_lonlat.csv'
df = pd.read_csv(csv_file)
sorted_df = df.sort_values(by="idle_time", ascending=False)
print(sorted_df.info())
sorted_df['end_station_lat'] = sorted_df['end_station_lat'].str[1:-1].astype(np.float64)
sorted_df['end_station_lon'] = sorted_df['end_station_lon'].str[1:-1].astype(np.float64)
sorted_df['idle_time'] = pd.to_timedelta(sorted_df['idle_time'])
sorted_df = sorted_df[sorted_df['idle_time'] <= pd.Timedelta(days=30)]


sorted_df['idle_time'] = sorted_df['idle_time'].dt.total_seconds()/3600


nodups_df = sorted_df.drop_duplicates(subset=["end_station_lon", "end_station_lat"], keep = "first")
print(len(sorted_df))
top50_percent = sorted_df.head(197)
bottom50_percent = sorted_df.tail(196)
top50_percent.info(), bottom50_percent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394 entries, 0 to 80
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   end_station_id   394 non-null    int64 
 1   idle_time        394 non-null    object
 2   morgens          394 non-null    int64 
 3   vormittags       394 non-null    int64 
 4   mittags          394 non-null    int64 
 5   nachmittags      394 non-null    int64 
 6   abends           394 non-null    int64 
 7   nachts           394 non-null    int64 
 8   monday           394 non-null    int64 
 9   tuesday          394 non-null    int64 
 10  wednesday        394 non-null    int64 
 11  thursday         394 non-null    int64 
 12  friday           394 non-null    int64 
 13  saturday         394 non-null    int64 
 14  sunday           394 non-null    int64 
 15  end_station_lat  394 non-null    object
 16  end_station_lon  394 non-null    object
dtypes: int64(14), object(3)
memory usage

(None, None)

In [37]:
avg_idle_time_top_50 = top50_percent["idle_time"].mean()
avg_idle_time_bottom_50 = bottom50_percent["idle_time"].mean()

over_top_avg_df = sorted_df[sorted_df["idle_time"] >= avg_idle_time_top_50]
# over_top_avg_df.count()

under_bottom_avg_df = sorted_df[sorted_df["idle_time"] <= avg_idle_time_bottom_50]


In [38]:
from geopy import distance
over_top_avg_df["neighboring_stations"] = None
over_top_avg_df["avg_neighbor_idle_time"] = None
for i, row in over_top_avg_df.iterrows():
    station_id = i
    lat = row['end_station_lat']
    lon = row['end_station_lon']
    center_coords = (lat, lon)
    neighbor_idle_time_sum = pd.Timedelta(0)


    neighboring_stations = []
    for index, sorted_row in sorted_df.iterrows():
        station_coords = (sorted_row['end_station_lat'], sorted_row['end_station_lon'])
        dist = distance.distance(center_coords, station_coords).meters

        if dist <= 500:
            neighboring_stations.append(sorted_row["end_station_id"])
            neighbor_idle_time_sum += pd.Timedelta(sorted_row["idle_time"])
    over_top_avg_df.at[i, "neighboring_stations"] = neighboring_stations



under_bottom_avg_df["neighboring_stations"] = None
under_bottom_avg_df["avg_neighbor_idle_time"] = None

for i, row in under_bottom_avg_df.iterrows():
    station_id = i
    lat = row['end_station_lat']
    lon = row['end_station_lon']
    center_coords = (lat, lon)
    neighbor_idle_time_sum = pd.Timedelta(0)


    neighboring_stations = []
    for index, sorted_row in sorted_df.iterrows():
        station_coords = (sorted_row['end_station_lat'], sorted_row['end_station_lon'])
        dist = distance.distance(center_coords, station_coords).meters

        if dist <= 500:
            neighboring_stations.append(sorted_row["end_station_id"])
            neighbor_idle_time_sum += pd.Timedelta(sorted_row["idle_time"])
    under_bottom_avg_df.at[i, "neighboring_stations"] = neighboring_stations


C:\Users\yusuf\AppData\Local\Temp\ipykernel_98632\2836388722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  over_top_avg_df["neighboring_stations"] = None
C:\Users\yusuf\AppData\Local\Temp\ipykernel_98632\2836388722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  over_top_avg_df["avg_neighbor_idle_time"] = None
C:\Users\yusuf\AppData\Local\Temp\ipykernel_98632\2836388722.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [39]:

over_top_avg_df.head()

,end_station_id,idle_time,morgens,vormittags,mittags,nachmittags,abends,nachts,monday,tuesday,wednesday,thursday,friday,saturday,sunday,end_station_lat,end_station_lon,neighboring_stations,avg_neighbor_idle_time
0,3000,221.201909,24162,23653,22421,21080,21352,24092,16488,16587,16570,16895,16961,17023,18160,34.048500,-118.258537,"[3000.0, 4286.0, 4516.0, 4285.0, 3063.0, 3032....",None
67,4125,154.930928,106,111,105,106,100,98,65,74,67,67,72,63,71,33.766659,-118.261017,[4125.0],None
293,4499,103.947705,124,132,120,124,122,122,74,75,72,85,82,72,85,34.186569,-118.388008,[4499.0],None
197,4379,101.813975,74,72,66,58,57,67,39,39,36,34,33,45,44,34.017368,-118.420601,"[4379.0, 4338.0, 4328.0, 4560.0, 4561.0, 4570.0]",None
202,4384,93.880557,133,131,116,117,86,118,62,64,69,58,59,73,55,33.990318,-118.412048,"[4384.0, 4337.0, 4577.0, 4521.0]",None


In [40]:
#Convert idle_time in df_sorted from string to timedelta
sorted_df['idle_time'] = pd.to_timedelta(sorted_df['idle_time'])

def calculate_avg_neighbor_idle_time(row):
    # Get the idle times of the neighboring stations from df_sorted
    neighbor_idle_times = sorted_df.loc[sorted_df['end_station_id'].isin(row['neighboring_stations']), 'idle_time']

    # Calculate and return the average idle time
    return neighbor_idle_times.mean()

over_top_avg_df['avg_neighbor_idle_time'] = over_top_avg_df.apply(calculate_avg_neighbor_idle_time, axis=1)
under_bottom_avg_df['avg_neighbor_idle_time'] = under_bottom_avg_df.apply(calculate_avg_neighbor_idle_time, axis=1)

over_top_avg_df.head(), under_bottom_avg_df.head()

C:\Users\yusuf\AppData\Local\Temp\ipykernel_98632\75913379.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  over_top_avg_df['avg_neighbor_idle_time'] = over_top_avg_df.apply(calculate_avg_neighbor_idle_time, axis=1)
C:\Users\yusuf\AppData\Local\Temp\ipykernel_98632\75913379.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  under_bottom_avg_df['avg_neighbor_idle_time'] = under_bottom_avg_df.apply(calculate_avg_neighbor_idle_time, axis=1)


(     end_station_id   idle_time  morgens  vormittags  mittags  nachmittags  \
 0              3000  221.201909    24162       23653    22421        21080   
 67             4125  154.930928      106         111      105          106   
 293            4499  103.947705      124         132      120          124   
 197            4379  101.813975       74          72       66           58   
 202            4384   93.880557      133         131      116          117   
 
      abends  nachts  monday  tuesday  wednesday  thursday  friday  saturday  \
 0     21352   24092   16488    16587      16570     16895   16961     17023   
 67      100      98      65       74         67        67      72        63   
 293     122     122      74       75         72        85      82        72   
 197      57      67      39       39         36        34      33        45   
 202      86     118      62       64         69        58      59        73   
 
      sunday  end_station_lat  end_station